In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [ ]:
loaders = {
    'train': DataLoader (train_data,
                         batch_size = 100,
                         shuffle = True,
                         num_workers = 1),
    
    'test': DataLoader(test_data,
                       batch_size = 100,
                       shuffle = True,
                       num_workers = 1)

}

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Increase the number of filters and layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4_drop = nn.Dropout2d()
        
        # Increase the number of neurons in the fully connected layers
        self.fc1 = nn.Linear(256 * 1 * 1, 1024)  # Updated to match the flattened output size
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        
        x = x.view(-1, 256 * 1 * 1)  # Flatten the tensor
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        
        return F.softmax(x, dim=1)

In [ ]:
# Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model, optimizer, and loss function
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Training function
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders["train"]):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\tLoss: {loss.item():.6f}')

# Testing function
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders["test"]:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(loaders["test"].dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)\n")

In [ ]:
for epoch in range(1, 3):
    train(epoch)
    test()

In [3]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import torch

# Device configuration (assuming you have a GPU setup, else set to 'cpu')
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# Initialize the Tkinter window
root = tk.Tk()
root.title("Draw a digit")

# Set up the canvas
canvas_size = 280  # Larger canvas size for easier drawing
drawing_size = 28  # Size to scale down to for prediction
pixel_size = canvas_size // drawing_size  # Each pixel on the 28x28 grid will be drawn as a 10x10 block

canvas = tk.Canvas(root, width=canvas_size, height=canvas_size, bg="white")
canvas.pack()

# Create an empty image for drawing
image = Image.new("L", (drawing_size, drawing_size), color=255)
draw = ImageDraw.Draw(image)

# Function to draw on the canvas
def paint(event):
    # Calculate the position on the 28x28 grid
    x = event.x // pixel_size
    y = event.y // pixel_size
    
    # Draw the "pixel" on the larger canvas
    canvas.create_rectangle(x * pixel_size, y * pixel_size, (x + 1) * pixel_size, (y + 1) * pixel_size, fill="black")
    
    # Draw on the smaller 28x28 image
    draw.rectangle([x, y, x + 1, y + 1], fill=0)

# Bind the paint function to the left mouse button
canvas.bind("<B1-Motion>", paint)

# Function to preprocess the image and make a prediction
def predict():
    # Invert the image (black background)
    inverted_image = ImageOps.invert(image)
    
    # Convert the image to a numpy array and normalize it
    image_array = np.array(inverted_image) / 255.0
    image_array = image_array.astype(np.float32)
    image_array = image_array[np.newaxis, np.newaxis, :, :]  # Add batch and channel dimensions

    # Convert the numpy array to a PyTorch tensor
    input_tensor = torch.from_numpy(image_array).to(device)
    
    # Make a prediction using the neural network
    with torch.no_grad():
        output = model(input_tensor)
        prediction = F.softmax(output, dim=1).argmax(dim=1).item()

    # Display the prediction
    result_label.config(text=f"Prediction: {prediction}")

def clear():
    canvas.delete("all")

# Button to make a prediction
predict_button = tk.Button(root, text="Predict", command=predict)
predict_button.pack()

restart_button = tk.Button(root, text="Clear Canvas", command=clear)
restart_button.pack()

# Label to display the result
result_label = tk.Label(root, text="Prediction: ")
result_label.pack()

# Start the Tkinter event loop
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\tester\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\tester\AppData\Local\Temp\ipykernel_1172\2907917953.py", line 56, in predict
    output = model(input_tensor)
             ^^^^^
NameError: name 'model' is not defined
